In [ ]:
Подготовка к собеседованию

In [ ]:
join - это тип объединения таблиц
INNER JOIN - берет только те строки, которые имеют совпадение по ключу в обеих таблицах.
в случае совпадений в обеих таблицах - 
Будет создаваться комбинация строк из обеих таблиц.
Если в правой таблице несколько совпадений для одной строки левой таблицы, то левые строки размножаются.
Это не совсем дубль, а декартово произведение совпадающих строк.

Декартово произведение (CROSS JOIN) — 
это объединение каждой строки первой таблицы с каждой строкой второй таблицы.

    
LEFT JOIN - 
Если берем поле из левой таблицы 
→ все строки левой таблицы сохраняются, NULL не будет.
Если берем поле из правой таблицы 
→ отображаются только совпавшие значения, 
а строки без совпадений заменяются NULL.


RIGHT JOIN - аналогично, как для left, но для правой таблицы 
FULL JOIN - Если есть совпадение — строки соединяются. 
Если в одной таблице строки без пары, то в другой подставляется NULL.

In [ ]:
Что такое индекс, какие типы индексов есть в PostgreSQL, как они работают?
Кластерный - 1 на таблицу, создается, чтобы организовать порядок строк и облегчить выборку. значения в полях должны быть уникальны. можно несколько полей. При создании кластерного индекса создается структура в БД
Noncluster - можно несколько полей, создающих порядок строк, для облегчения выборки. не занимает в БД место
Покрывной индекс - это выборка полей, которые часто используются в запросе, то есть покрытие таблицы структурой, для ускорения вывода данных




In [1]:
про Explain и Explain Analyze хорошо рассказано тут 
https://www.youtube.com/watch?v=bzCqU1gSsM0

SyntaxError: invalid syntax (3203103757.py, line 1)

In [2]:
позволяет посмотреть схему выполнения запроса, время выполнения и применяется ли индекс или нет

Explore - показывает как бы теорию, время выполнения запроса, а Explain Analyze показывает реальное выполнение

для обычных запросов обычно строят btree индекс, для всяких более гибких запросов, вроде использование like '%abc%' можно использовать gin индекс

подключение gin индекса:

важно уесть, что индекс не обяхан использоваться
если выборка большая, система может решить, что ей выгоднее использовать построчный поиск


SyntaxError: invalid syntax (169757806.py, line 1)

In [ ]:
CREATE EXTENSION pg_trgm

select * 
from perf_test
where reason like '%bc%'
;

create index trgm_idx_perf_reason on perf_test using gin (reason gin_trgm_ops)

In [ ]:
CTE - это сommon table expression
позволяет сделать рекурсивный запрос (
    with RECURSIVE as t ()
)

 либо вывести запрос наружу, если он несколько раз используется в общем запросе

CTE полезен при 
1) Переборе значений в каталогах, маршрутах, иерархии
WITH top_users AS (
    SELECT id, name, score 
    FROM users 
    WHERE score > 1000
)
SELECT * FROM top_users ORDER BY score DESC;

2) Для обхода ограничений на подзапросы (например, в UPDATE и DELETE).
WITH RECURSIVE category_tree AS (
    -- Базовый случай (верхний уровень каталога)
    SELECT id, parent_id, name
    FROM categories
    WHERE parent_id IS NULL
    UNION ALL
    -- Рекурсивное соединение с самим собой
    SELECT c.id, c.parent_id, c.name
    FROM categories c
    JOIN category_tree ct ON c.parent_id = ct.id
)
SELECT * FROM category_tree;




Обновим цены товаров, которые входят в популярные категории (id > 10)

sql
Copy
Edit
WITH popular_products AS (
    SELECT id, price * 1.1 AS new_price
    FROM products
    WHERE category_id > 10
)
UPDATE products 
SET price = popular_products.new_price
FROM popular_products
WHERE products.id = popular_products.id;


 Удалим пользователей, которые не заходили больше года

sql
Copy
Edit
WITH inactive_users AS (
    SELECT id FROM users WHERE last_login < NOW() - INTERVAL '1 year'
)
DELETE FROM users 
USING inactive_users 
WHERE users.id = inactive_users.id;

In [ ]:
DROP – полностью удаляет таблицу, включая её структуру. ROLLBACK НЕ РАБОТАЕТ
TRUNCATE – быстро очищает все строки таблицы, но не удаляет саму таблицу, индексы. ROLLBACK работает
DELETE – удаляет строки по условию (WHERE) или все строки, но медленнее TRUNCATE. ROLLBACK работает



    

In [ ]:
MVCC (Multiversion Concurrency Control) в PostgreSQL
Как это работает?

Каждая запись имеет две метки времени (xmin и xmax).
При UPDATE новая версия строки создаётся, а старая остаётся в таблице (но "невидима" для новых транзакций).
DELETE не удаляет строки сразу, а просто помечает их "невидимыми" (xmax)

In [ ]:
Как работает VACUUM в PostgreSQL

Postgresql - при операциях UPDATE, DELETE не происходит физического удаления места в таблице.
команда: 
    VACUUM table - помечает эти "мусорные" строки к перезаписыванию. Таблица не блокируется
    VACUUM FULL table - происходит пересоздание таблицы, то есть она блокируется (чтение, запись), появляется ее дубль соразмерныей ее объему.
    VACUUM ANALYZER - это позволяет пометить строки и переписать план выполнения запроса SQL


In [ ]:
🔹 Чем GROUP BY отличается от PARTITION BY?

 GROUP BY - это группировка агрегированных данных (sum, count)

SELECT department, COUNT(*) 
FROM employees 
GROUP BY department;

 PARTITION BY - это разбивка данных внутри оконных функций

SELECT 
    department, 
    employee_name, 
    COUNT(*) OVER (PARTITION BY department) AS total_employees
FROM employees;

In [ ]:
Как работает транзакция в PostgreSQL? Какие уровни изоляции транзакций существуют?

Транзакция - это единый запрос для выполнения операции
свойства ACID
Атомарность
Целостность
Изоляция
Надежность

BEGIN;
UPDATE accounts SET balance = balance - 100 WHERE id = 1;
UPDATE accounts SET balance = balance + 100 WHERE id = 2;
COMMIT;

ТС можно откатить

BEGIN;
UPDATE accounts SET balance = balance - 100 WHERE id = 1;
ROLLBACK; -- Отмена изменений


Уровни транзакций
READ COMMITTED (по умолчанию в PostgreSQL) - Каждый новый SELECT видит актуальные данные. 
REPEATABLE READ — «замораживает» данные. Даже если кто-то в другой транзакции внёс изменения и сделал COMMIT, мы этого не увидим, пока не закончим свою транзакцию.
SERIALIZABLE — самый строгий уровень. Имитация последовательного выполнения транзакций (будто они выполнялись одна за другой). Если возможен конфликт → одна из транзакций отменяется (ROLLBACK).

READ COMMITTED
Когда применять?
✅ Когда важна актуальность данных, но не критично, что результаты могут немного отличаться в течение транзакции.

REPEATABLE READ
Когда применять?
✅ Когда важно, чтобы данные не менялись в течение транзакции, например, при генерации отчётов или расчёте среднего чека.

SERIALIZABLE
Когда применять?
✅ Когда нужна 100% защита от конфликтов, например, при финансовых операциях.
⚠ Минус: может приводить к частым ROLLBACK, если много параллельных операций.


In [ ]:
READ COMMITTED 
-- Транзакция 1
BEGIN;
UPDATE users SET balance = balance - 500 WHERE id = 1;
-- Пока нет COMMIT, другие транзакции не видят изменения

-- Транзакция 2
BEGIN;
SELECT balance FROM users WHERE id = 1; -- Видит старые данные, пока не будет COMMIT в Транзакции 1
COMMIT;

REPEATABLE READ
-- Транзакция 1
BEGIN TRANSACTION ISOLATION LEVEL REPEATABLE READ;
SELECT SUM(balance) FROM users; -- 10000

-- Транзакция 2 (вносит изменения и фиксирует их)
BEGIN;
UPDATE users SET balance = balance - 500 WHERE id = 1;
COMMIT;

-- Транзакция 1 снова делает SELECT (видит старые данные!)
SELECT SUM(balance) FROM users; -- Всё ещё 10000
COMMIT;

SERIALIZABLE
-- Транзакция 1
BEGIN TRANSACTION ISOLATION LEVEL SERIALIZABLE;
SELECT balance FROM users WHERE id = 1; -- 5000

-- Транзакция 2 изменяет то же значение
BEGIN TRANSACTION ISOLATION LEVEL SERIALIZABLE;
UPDATE users SET balance = balance - 100 WHERE id = 1;
COMMIT;

-- Транзакция 1 пытается обновить, но получит ошибку
UPDATE users SET balance = balance - 200 WHERE id = 1;
COMMIT; -- ❌ Ошибка: транзакция конфликтует!

In [ ]:
Что такое NULL и как с ним работать в ORDER BY, GROUP BY, JOIN

NULL - в постгрес это отдельный тип данных, поэтому его можно группировать

ORDER BY - ASC - NULL вверху, DESC - NULL внизу
GROUP BY - при группировке покажет кол-во NULL в табл
JOIN - не объединяет их. но можно делать отбор 
where s in NULL
where s is not null

In [ ]:
Напишите SQL-запрос, который вернет:
Количество заказов (count(user_id), sum(amount)) и суммарную сумму заказов для каждого user_id,
Только для пользователей, у которых сумма всех заказов больше 500.

In [ ]:
CREATE TABLE orders (
    id SERIAL PRIMARY KEY,
    user_id INT,
    order_date DATE,
    amount DECIMAL
);

INSERT INTO orders (user_id, order_date, amount) VALUES
(1, '2024-03-01', 200),
(1, '2024-03-02', 100),
(1, '2024-03-05', 300),
(2, '2024-03-01', 150),
(2, '2024-03-02', 100),
(3, '2024-03-01', 600),
(3, '2024-03-02', 50);

select
user_id,
count(*),
sum(amount)
from orders
group by user_id
having sum(amount) > 500;


In [ ]:
CREATE TABLE users (
    id SERIAL PRIMARY KEY,
    name TEXT
);

CREATE TABLE orders (
    id SERIAL PRIMARY KEY,
    user_id INT REFERENCES users(id),
    order_date DATE,
    amount DECIMAL
);

INSERT INTO users (id, name) VALUES
(1, 'Alice'),
(2, 'Bob'),
(3, 'Charlie'),
(4, 'David'); -- David не сделал заказов

INSERT INTO orders (user_id, order_date, amount) VALUES
(1, '2024-03-01', 200),
(2, '2024-03-02', 150),
(3, '2024-03-03', 300);

/* Найти пользователей без заказов */

select 
u.name,
count(amount) as cnt_ord
from users u
left join orders o on o.user_id = u.id
where o.id is null
group by u.name
;

In [ ]:
CREATE TABLE users (
    id SERIAL PRIMARY KEY,
    name TEXT
);

CREATE TABLE orders (
    id SERIAL PRIMARY KEY,
    user_id INT REFERENCES users(id),
    order_date DATE,
    amount DECIMAL
);

INSERT INTO users (name) VALUES 
('Alice'),
('Bob'),
('Charlie');

INSERT INTO orders (user_id, order_date, amount) VALUES 
(1, '2024-03-01', 100),  
(1, '2024-03-05', 250),  
(2, '2024-03-10', 50),  
(2, '2024-03-12', 300),  
(3, '2024-03-15', 100);  


/* Найти пользователей, у которых средняя сумма заказов больше 150*/

select user_id,
round(avg(amount),0) as avg_o
from
orders
group by user_id
having avg(amount) > 150;


/* Задача 2: Найти заказы, которые дороже среднего чека всех заказов
*/
-- находим средний чек
with cte as (
select 
round(avg(amount), 0) as avg_o
from orders
  )
-- сравниваем со всеми  
  select * 
  from orders
  cross join cte
  where amount > avg_o


In [ ]:
/*  Найти всех пользователей и их сумму заказов, включая тех, у кого заказов нет
*/


select 
    u.id,
    u.name,
    sum(amount) as s_buy
from users u
    left join orders o
on o.user_id = u.id
group by u.id,
        u.name


In [ ]:
/*  Задача 4: Найти клиентов, которые делали заказы в разные дни */

SELECT user_id, 
       COUNT(DISTINCT order_date) AS unique_days
FROM orders
GROUP BY user_id
HAVING COUNT(DISTINCT order_date) > 1;



In [ ]:
 Вывести всех пользователей и их заказы, но если заказов нет — указать 
"Нет заказов"


select 
u.name,
coalesce (o.amount::TEXT, "Нет заказов")
    cast
as cnt
 from users left join orders ON u.id = o.user_id;

In [ ]:
/* Вывести всех пользователей и их заказы, но если заказов нет — указать 'Нет заказов'
*/


select 
u.name,
CASE 
	when o.amount is null THEN 'Нет заказов'
    else o.amount::TEXT
END as order_amount,
coalesce (o.amount::TEXT, 'Нет заказов') as cnt
from users u left join orders o 
 ON u.id = o.user_id;

In [ ]:

Задачи (по WITH AS и UNION | UNION ALL):
WITH AS
1️⃣ Найти среднюю сумму заказов по каждому клиенту и выбрать клиентов с суммой покупок выше 500.



    SELECT
        o.customer_id,
        SUM(p.price * oi.quantity) AS total_buy
    FROM orders o
    JOIN order_items oi ON oi.order_id = o.id
    JOIN products p ON p.id = oi.product_id
    GROUP BY o.customer_id
  having SUM(p.price * oi.quantity) > 500;

*/


/*
Определить количество проданных товаров по категориям и вывести категории, где продано более 5 товаров.
*/

SELECT 
c.name,
sum(oi.quantity) as sum_sell
FROM orders o
    JOIN order_items oi ON oi.order_id = o.id
    JOIN products p ON p.id = oi.product_id
	JOIN categories c ON p.category_id = c.id
 group by  c.name
having sum(oi.quantity) > 5;



3 Найти самую дорогую покупку каждого клиента.
WITH t AS (
    SELECT 
        o.customer_id,
        c.name,
        oi.order_id,
        SUM(oi.quantity * p.price) AS sum_buy_per_user
    FROM orders o
    JOIN order_items oi ON oi.order_id = o.id
    JOIN products p ON p.id = oi.product_id
    JOIN customers c ON o.customer_id = c.id
    GROUP BY o.customer_id, c.name, oi.order_id    
)
SELECT 
    customer_id,
    name,
    MAX(sum_buy_per_user) AS max_buy
FROM t
GROUP BY customer_id, name;

In [ ]:
UNION

1 Вывести список всех клиентов и всех категорий товаров в одном списке.
*/

SELECT
c.name as clients_and_cat,
'Clients' as type
FROM customers c
union all
SELECT
cat.name as clients_and_cat,
'Category' as type
FROM categories cat

In [ ]:
Найти клиентов, которые покупали и Electronics, и Clothing.
*/
SELECT 
c.name
FROM orders o
    JOIN order_items oi ON oi.order_id = o.id
    JOIN products p ON p.id = oi.product_id
	JOIN categories cat ON p.category_id = cat.id
    JOIN customers c ON o.customer_id = c.id
where cat.name in ('Electronics', 'Clothing')
group by
c.name
having count(dictinct c.name) =2


In [1]:
union 

-- Создание таблицы клиентов
CREATE TABLE customers (
    id SERIAL PRIMARY KEY,
    name TEXT,
    age INT
);

-- Создание таблицы заказов
CREATE TABLE orders (
    id SERIAL PRIMARY KEY,
    customer_id INT REFERENCES customers(id),
    order_date DATE,
    total_amount DECIMAL(10,2)
);

-- Создание таблицы товаров
CREATE TABLE products (
    id SERIAL PRIMARY KEY,
    name TEXT,
    category TEXT,
    price DECIMAL(10,2)
);

-- Заполнение данными (клиенты)
INSERT INTO customers (name, age) VALUES
('Alice', 25),
('Bob', 40),
('Charlie', 30),
('Diana', 35);

-- Заполнение данными (заказы)
INSERT INTO orders (customer_id, order_date, total_amount) VALUES
(1, '2024-03-10', 120.50),
(2, '2024-03-11', 300.00),
(1, '2024-03-15', 80.75),
(3, '2024-03-12', 150.00);

-- Заполнение данными (товары)
INSERT INTO products (name, category, price) VALUES
('Laptop', 'Electronics', 1200.00),
('T-Shirt', 'Clothing', 20.00),
('Jeans', 'Clothing', 50.00),
('Smartphone', 'Electronics', 800.00),
('Book', 'Books', 15.00);

NameError: name 'union' is not defined

In [ ]:
-- Вывести список всех клиентов и всех категорий товаров в одном общем списке.

select 
name,
'clients' as type
from customers
union
select 
category,
'category' as type
from products

In [ ]:
-- Вывести заказы клиентов старше 30 лет ИЛИ с суммой заказа больше 200.

select 
o.id, c.name, o.order_date, o.total_amount
from orders o
left join customers c on o.customer_id = c.id
where age > 30 or total_amount > 200;

In [ ]:
Вывести товары с пометкой:

"Expensive" если цена выше 500
"Affordable" если от 100 до 500
"Cheap" если меньше 100

select 
name, category, price,
CASE 
WHEN price > 500
THEN 'Expensive'

WHEN price > 100
THEN 'Affordable'

WHEN price < 100
THEN 'Cheap'
END

from products
;

In [ ]:
-- Вывести заказы, но сумму заказов преобразовать в строку.
select 
customer_id, order_date, 
CAST(total_amount as TEXT) as price
from orders
;